Scraping, summarizing, and utilizing DL concepts in order to gain insights on stock news

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.5MB/s 
     |████████████████████████████████| 3.3MB 38.8MB/s 
     |████████████████████████████████| 901kB 30.8MB/s 


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 8.4MB/s 


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [ ]:
name_model = "human-centered-summarization/financial-summarization-pegasus" #financial model from website
tokenizer = PegasusTokenizer.from_pretrained(name_model) #passing to model and tokenizer. #essentially translator for nlp
model = PegasusForConditionalGeneration.from_pretrained(name_model)#loaded in model

summarizing single article to show proof of concept

In [ ]:
url = "https://finance.yahoo.com/news/bitcoin-volatility-decline-paves-way-212136332.html"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
all_paragraphs = soup.find_all('p') #find all paragraphs in article
all_paragraphs # need to clean to a good format

[<p>(Bloomberg) -- The recent pullback in Bitcoin’s volatility is setting the stage for a trend that could encourage institutions to dive in, according to JPMorgan Chase &amp; Co.</p>,
 <p>“These tentative signs of Bitcoin volatility normalization are encouraging,” strategists including Nikolaos Panigirtzoglou wrote in report emailed Thursday. “In our opinion, a potential normalization of Bitcoin volatility from here would likely help to reinvigorate the institutional interest going forward.”</p>,
 <p>Three-month realized volatility for the cryptocurrency has fallen to 86% after rising above 90% in February, they wrote. The six-month measure appears to be stabilizing at around 73%. As volatility subsides, a greater number of institutions could warm to the crypto space, the strategists said.</p>,
 <p>The coin’s volatility has kept institutions away, something that’s been a key consideration for risk management -- the higher the volatility of an asset, the higher the risk capital consume

In [ ]:
all_paragraphs[0].text #first paragraph

'(Bloomberg) -- The recent pullback in Bitcoin’s volatility is setting the stage for a trend that could encourage institutions to dive in, according to JPMorgan Chase & Co.'

In [ ]:
all_text = [paragraph.text for paragraph in all_paragraphs]
words = ' '.join(all_text).split(' ')[:400] #have to cap as there is a limit. Also joining texts into one string and then splitting words. Grabbing first 450 words
article = ' '.join(words)

In [ ]:
print(words)
len(words) #400

['(Bloomberg)', '--', 'The', 'recent', 'pullback', 'in', 'Bitcoin’s', 'volatility', 'is', 'setting', 'the', 'stage', 'for', 'a', 'trend', 'that', 'could', 'encourage', 'institutions', 'to', 'dive', 'in,', 'according', 'to', 'JPMorgan', 'Chase', '&', 'Co.', '“These', 'tentative', 'signs', 'of', 'Bitcoin', 'volatility', 'normalization', 'are', 'encouraging,”', 'strategists', 'including', 'Nikolaos', 'Panigirtzoglou', 'wrote', 'in', 'report', 'emailed', 'Thursday.', '“In', 'our', 'opinion,', 'a', 'potential', 'normalization', 'of', 'Bitcoin', 'volatility', 'from', 'here', 'would', 'likely', 'help', 'to', 'reinvigorate', 'the', 'institutional', 'interest', 'going', 'forward.”', 'Three-month', 'realized', 'volatility', 'for', 'the', 'cryptocurrency', 'has', 'fallen', 'to', '86%', 'after', 'rising', 'above', '90%', 'in', 'February,', 'they', 'wrote.', 'The', 'six-month', 'measure', 'appears', 'to', 'be', 'stabilizing', 'at', 'around', '73%.', 'As', 'volatility', 'subsides,', 'a', 'greater', 

400

In [ ]:
article

'(Bloomberg) -- The recent pullback in Bitcoin’s volatility is setting the stage for a trend that could encourage institutions to dive in, according to JPMorgan Chase & Co. “These tentative signs of Bitcoin volatility normalization are encouraging,” strategists including Nikolaos Panigirtzoglou wrote in report emailed Thursday. “In our opinion, a potential normalization of Bitcoin volatility from here would likely help to reinvigorate the institutional interest going forward.” Three-month realized volatility for the cryptocurrency has fallen to 86% after rising above 90% in February, they wrote. The six-month measure appears to be stabilizing at around 73%. As volatility subsides, a greater number of institutions could warm to the crypto space, the strategists said. The coin’s volatility has kept institutions away, something that’s been a key consideration for risk management -- the higher the volatility of an asset, the higher the risk capital consumed by it, according to the strategi

In [ ]:
input_ids = tokenizer.encode(article, return_tensors = 'pt') #article into encoded seq
output = model.generate(input_ids, max_length=100, early_stopping = True) #ids into model
summary = tokenizer.decode(output[0], skip_special_tokens = True)

In [ ]:
summary #now created a short summary of article

'‘ tentative signs of Bitcoin volatility normalization’: strategists. Goldman close to offering crypto funds to private wealth clients'

Building a monitor system for multiple tickers

In [ ]:
mon_tickers = ['TSLA','HVBTF']

In [ ]:
def stock_news_search(ticker):
  search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker) #set up base, with ticker of interest filling in {}
  r = requests.get(search_url)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all('a')
  hrefs = [link['href'] for link in atags]
  return hrefs

In [ ]:
urls_all = {ticker:stock_news_search(ticker) for ticker in mon_tickers}#dict, setting key as ticker
urls_all

{'HVBTF': ['/?sa=X&ved=0ahUKEwiNgrTRk5XwAhVKo54KHXF5DKgQOwgC',
  '/?output=search&ie=UTF-8&tbm=nws&sa=X&ved=0ahUKEwiNgrTRk5XwAhVKo54KHXF5DKgQPAgE',
  '/search?q=yahoo+finance+HVBTF&tbm=nws&ie=UTF-8&gbv=1&sei=BSeDYM3rI8rG-gTx8rHACg',
  '/search?q=yahoo+finance+HVBTF&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiNgrTRk5XwAhVKo54KHXF5DKgQ_AUIBygA',
  '/search?q=yahoo+finance+HVBTF&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwiNgrTRk5XwAhVKo54KHXF5DKgQ_AUICSgC',
  '/search?q=yahoo+finance+HVBTF&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiNgrTRk5XwAhVKo54KHXF5DKgQ_AUICigD',
  '/search?q=yahoo+finance+HVBTF&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiNgrTRk5XwAhVKo54KHXF5DKgQ_AUICygE',
  'https://maps.google.com/maps?q=yahoo+finance+HVBTF&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiNgrTRk5XwAhVKo54KHXF5DKgQ_AUIDCgF',
  '/search?q=yahoo+finance+HVBTF&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiNgrTRk5XwAhVKo54KHXF5DKgQ_AUIDSgG',
  '/advanced_search',
  '/search?q=yahoo+finance+HVBTF&ie=UTF-8&tbm=nws&sourc

In [ ]:
urls_all.keys() #tickers mentioned in mon_tickers (keys to dict)

dict_keys(['TSLA', 'HVBTF'])

In [ ]:
import re

In [ ]:
#want to remove unwanted/weird links
list_to_exclude = ['maps','policies','preferences','accounts','support']

def strip_urls(urls_all, list_to_exclude):
  val = [] #creating array to store temp results
  for url in urls_all:
    if 'https://' in url and not any(exclude_word in url for exclude_word in list_to_exclude):
      res = re.findall(r'(https://\S+)',url)[0].split('&')[0]
      val.append(res)
  return list(set(val)) # set erases duplicates

In [ ]:
strip_urls(urls_all['TSLA'], list_to_exclude) #clean urls

['https://finance.yahoo.com/news/teslas-market-share-was-probably-close-to-peak-in-2020-us-ev-analyst-102439264.html',
 'https://finance.yahoo.com/news/smart-money-piling-tesla-inc-141938791.html',
 'https://finance.yahoo.com/news/tesla-ceo-musk-puts-100-213327743.html',
 'https://finance.yahoo.com/news/forget-tesla-cars-real-money-175911310.html',
 'https://finance.yahoo.com/news/us-stocks-wall-street-slips-202001760.html',
 'https://finance.yahoo.com/news/why-earnings-season-could-great-124012518.html',
 'https://finance.yahoo.com/news/tesla-inc-tsla-cfo-zachary-031502365.html',
 'https://finance.yahoo.com/news/tesla-comes-under-growing-china-163458554.html',
 'https://finance.yahoo.com/news/tesla-tsla-gains-lags-market-214509336.html',
 'https://finance.yahoo.com/news/tesla-tsla-post-solid-q1-141902272.html']

In [ ]:
urls_c = {ticker:strip_urls(urls_all[ticker],list_to_exclude) for ticker in mon_tickers}
urls_c

{'HVBTF': ['https://ca.finance.yahoo.com/news/hive-stock-ride-emerging-trend-171259079.html',
  'https://finance.yahoo.com/news/hive-completes-share-investment-defi-050000414.html',
  'https://ca.finance.yahoo.com/news/alert-hive-blockchain-sees-bitcoin-153000741.html',
  'https://ca.finance.yahoo.com/news/think-bitcoin-bounce-hive-blockchain-120058026.html',
  'https://finance.yahoo.com/news/hive-blockchain-completes-acquisition-green-050000418.html',
  'https://finance.yahoo.com/news/defi-technologies-announces-strategic-partnership-071300152.html',
  'https://ca.finance.yahoo.com/news/ultimate-meme-stock-hive-blockchain-140041822.html',
  'https://ca.finance.yahoo.com/news/hive-blockchain-stock-could-rocket-160054739.html',
  'https://ca.finance.yahoo.com/news/hive-blockchain-stock-epic-rally-160001706.html',
  'https://ca.finance.yahoo.com/news/2-top-stocks-could-soar-123209287.html'],
 'TSLA': ['https://finance.yahoo.com/news/teslas-market-share-was-probably-close-to-peak-in-2020-

In [ ]:
def scraping_text(URLS):
  articles = []
  for url in URLS:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all('p')
    all_text = [paragraph.text for paragraph in paragraphs]
    words = ' '.join(all_text).split(' ')[:400] #have to cap as there is a limit. Also joining texts into one string and then splitting words. Grabbing first 450 words
    article = ' '.join(words)
    articles.append(article)
  return articles

In [ ]:
ARTICLES = {ticker:scraping_text(urls_c[ticker]) for ticker in mon_tickers}
ARTICLES

{'HVBTF': ['New home sales surge to the highest level since 2006 in March Cryptocurrency miner Hive Blockchain Technologies (TSXV:HIVE) saw its stock price surge over 3,600% at one point over the past 12 months. Rising prices for Bitcoin and Ethereum powered Hive stock price surge last year. However, thanks to the company’s significant exposure to Ethereum, Hive’s stock price growth could be sustained by the rapid growth in decentralized finance (DeFi) applications this year. Hive Blockchain prides itself as the only publicly traded Ethereum mining stock for a good reason. It has invested in a blockchain platform that is gaining wider institutional business adoption at a faster rate than Bitcoin. The company has two large data centers in Sweden and Iceland that are 100% focused on Ethereum mining operations. Only one data center (the Canadian one) is mining for Bitcoin. Ethereum has a platform for “smart contracts” which has gained wide adoption in the financial transactions revolution

In [ ]:
len(ARTICLES['TSLA']) #exploring findings to ensure it worked
ARTICLES['TSLA']

['Tesla\'s (TSLA) record-setting share price rally in 2020 coincided with what may have been an all-time high in the company\'s share of the U.S. electric-vehicle market, according to at least one analyst.\xa0 "In the U.S., [Tesla has] more than quadrupled their market share in the U.S. electric vehicle market over the last five years," Garrett Nelson, CFRA analyst, told Yahoo Finance Live on Monday. "But we think their market share is probably pretty close to a peak in 2020."\xa0 "They just haven\'t had the competition, and they\'ve really benefited from that. That will be changing very soon. In fact, in the next six months or so, you will see Lucid and Rivian vehicles on the road. You will see a number of new models from traditional automakers in electric vehicles on the road as well. So it\'s really the increased competition [that] has us cautious on the stock here." With competition in the electric-vehicle space starting to heat up, the surge in Tesla\'s stock price will likely be 

In [ ]:
def summarize(ARTICLES):
    summaries = []
    for article in ARTICLES:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [ ]:
summaries = {ticker:summarize(ARTICLES[ticker]) for ticker in mon_tickers}
summaries

Token indices sequence length is longer than the specified maximum sequence length for this model (517 > 512). Running this sequence through the model will result in indexing errors


IndexError: ignored

In [ ]:
summaries['ETH']